In [20]:
import time
import oauth2
import tweepy
import pandas as pd
from tqdm import tqdm
from Twitter_codes import keys

In [21]:
CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET = keys()

In [22]:
def oauth_req(url, http_method="GET", post_body="", http_headers=None):
    consumer = oauth2.Consumer(key=CONSUMER_KEY, secret=CONSUMER_SECRET)
    token = oauth2.Token(key=ACCESS_TOKEN, secret=ACCESS_SECRET)
    client = oauth2.Client(consumer, token)
    resp, content = client.request( url, method=http_method, body=bytes(post_body, "utf-8"), headers=http_headers )
    return content

#home_timeline = oauth_req( 'https://api.twitter.com/1.1/statuses/home_timeline.json')

In [23]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

In [24]:
search = ['@McDonalds', 
         '@Sonicdrivein', 
         '@Starbucks',
         '@Shakeshack',
         '@Dominos',
         '@DennysDiner',
         '@ChipotleTweets',
         '@Wendys',
         '@Dunkindonuts']

In [25]:
date = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))

In [26]:
all_companies = []
for company in tqdm(search):
    list_tweets = []
    for tweet in tweepy.Cursor(api.search, q=company, retry_delay=5, wait_on_rate_limit=True, lang="en").items(500):
        if not tweet.retweeted:
            if not tweet.is_quote_status:
                if not tweet.user.screen_name == company[1:]:
                    twitter_dic = {}
                    twitter_dic['Company'] = company
                    twitter_dic['time_tweeted'] = tweet.created_at
                    twitter_dic['unique_code'] = tweet.id_str

                    twitter_dic['text'] = tweet.text
                    twitter_dic['meta_lang'] = tweet.metadata['iso_language_code']
                    twitter_dic['hashtag'] = tweet.entities['hashtags']
                    twitter_dic['source'] = tweet.source

                    twitter_dic['retweet_count'] = tweet.retweet_count     
                    twitter_dic['favorite_count'] = tweet.favorite_count
                    twitter_dic['user_followers_count'] = tweet.author.followers_count
                    twitter_dic['user_name'] = tweet.author.name
                    twitter_dic['user_location'] = tweet.author.location
                    twitter_dic['number_of_user_tweets'] = tweet.author.statuses_count
                    twitter_dic['user_is_verified'] = tweet.author.verified
                    twitter_dic['user_profile_text'] = tweet.author.description
                    twitter_dic['number_of_people_they_follow'] = tweet.author.friends_count
                    list_tweets.append(twitter_dic)
    all_companies.extend(list_tweets) 

100%|██████████| 9/9 [01:46<00:00, 11.85s/it]


In [72]:
df = pd.DataFrame(all_companies)

In [ ]:
#pdf = pd.DataFrame(columns=df.columns)
pdf = pd.read_csv('./Data/Tweets.csv')

In [74]:
pdf = pdf.append(df)

In [75]:
pdf.to_csv('./Data/Tweets.csv',index=False)